In [3]:
import os
from PIL import Image
from pathlib import Path

In [8]:
IMG_SIZE = (224, 224)
ORIG_BASE = "../input"
DEST_BASE = "../processed"


In [10]:
def process_images(subdir, is_test=False):
    input_dir = Path(ORIG_BASE) / subdir
    output_dir = Path(DEST_BASE) / subdir
    output_dir.mkdir(parents=True, exist_ok=True)

    if is_test:
        for img_path in input_dir.glob("*.*"):
            try:
                img = Image.open(img_path).convert("RGB")
                img = img.resize(IMG_SIZE, Image.LANCZOS)
                dest_path = output_dir / img_path.name
                img.save(dest_path)
            except Exception as e:
                print(f"Erro ao processar {img_path}: {e}")
    else:
        for class_folder in input_dir.iterdir():
            if class_folder.is_dir():
                output_class_dir = output_dir / class_folder.name
                output_class_dir.mkdir(parents=True, exist_ok=True)

                for img_path in class_folder.glob("*.jpg"):
                    try:
                        img = Image.open(img_path).convert("RGB")
                        img = img.resize(IMG_SIZE, Image.LANCZOS)
                        dest_path = output_class_dir / img_path.name
                        img.save(dest_path)
                    except Exception as e:
                        print(f"Erro ao processar {img_path}: {e}")

for subset in ["train", "valid", "test"]:
    print(f"Processando: {subset}")
    is_test = subset == "test"
    process_images(subset, is_test)

Processando: train
Processando: valid
Processando: test


In [25]:
BASE_DIR = os.path.abspath(os.path.join(os.getcwd(), '..'))
TRAIN_DIR = os.path.join(BASE_DIR, 'processed', 'train')
VALID_DIR = os.path.join(BASE_DIR, 'processed', 'valid')
TEST_DIR = os.path.join(BASE_DIR, 'processed', 'test')

train_ds = tf.keras.preprocessing.image_dataset_from_directory(
    TRAIN_DIR,
    image_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
    label_mode="int",
    color_mode="rgb",
    seed=123
)
for images, labels in train_ds.take(1):
    print(images.shape)

val_ds = tf.keras.preprocessing.image_dataset_from_directory(
    VALID_DIR,
    image_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
    label_mode="int",
    color_mode="rgb",
    seed=123
)

test_ds = tf.keras.preprocessing.image_dataset_from_directory(
    TEST_DIR,
    image_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
    label_mode="int",
    color_mode="rgb",
    seed=123
)


Found 6552 files belonging to 102 classes.
(32, 224, 224, 3)
Found 818 files belonging to 102 classes.
Found 819 files belonging to 1 classes.


In [ ]:
normalization_layer = tf.keras.layers.Rescaling(1./255)
data_augmentation = tf.keras.Sequential([
    tf.keras.layers.RandomFlip('horizontal'),
    tf.keras.layers.RandomRotation(0.2),
    tf.keras.layers.RandomZoom(0.2),
])

In [18]:
train_ds = train_ds.map(lambda x, y: (data_augmentation(x), y))

train_ds = train_ds.map(lambda x, y: (normalization_layer(x), y))
val_ds = val_ds.map(lambda x, y: (normalization_layer(x), y))
test_ds = test_ds.map(lambda x, y: (normalization_layer(x), y))


In [19]:
train_ds = train_ds.cache().batch(BATCH_SIZE).prefetch(buffer_size=tf.data.experimental.AUTOTUNE)
val_ds = val_ds.cache().batch(BATCH_SIZE).prefetch(buffer_size=tf.data.experimental.AUTOTUNE)
test_ds = test_ds.cache().batch(BATCH_SIZE).prefetch(buffer_size=tf.data.experimental.AUTOTUNE)

In [26]:
model = tf.keras.Sequential([
    tf.keras.layers.InputLayer(input_shape=(224, 224, 3)),
    tf.keras.layers.Rescaling(1./255),  # Normaliza as imagens entre 0 e 1
    tf.keras.layers.Conv2D(32, (3, 3), activation='relu'),
    tf.keras.layers.MaxPooling2D(),
    tf.keras.layers.Conv2D(64, (3, 3), activation='relu'),
    tf.keras.layers.MaxPooling2D(),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(128, activation='relu'),
    tf.keras.layers.Dense(102, activation='softmax')  # 102 classes para flores
])

C:\Users\pedro\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\keras\src\layers\core\input_layer.py:27: UserWarning: Argument `input_shape` is deprecated. Use `shape` instead.
  warnings.warn(


In [27]:
model.compile(
    optimizer='adam',
    loss='sparse_categorical_crossentropy',
    metrics=['accuracy']
)

In [28]:
# Verificar formato do primeiro batch de imagens
for images, labels in train_ds.take(1):
    print(images.shape)  # Verifique se a forma é (batch_size, altura, largura, canais)


(32, 224, 224, 3)


In [29]:
history = model.fit(
    train_ds,
    validation_data=val_ds,
    epochs=10
)

Epoch 1/10
205/205 ━━━━━━━━━━━━━━━━━━━━ 139s 670ms/step - accuracy: 0.0396 - loss: 5.4133 - val_accuracy: 0.1100 - val_loss: 3.7986
Epoch 2/10
205/205 ━━━━━━━━━━━━━━━━━━━━ 120s 586ms/step - accuracy: 0.1835 - loss: 3.4047 - val_accuracy: 0.2579 - val_loss: 3.0424
Epoch 3/10
205/205 ━━━━━━━━━━━━━━━━━━━━ 127s 619ms/step - accuracy: 0.4841 - loss: 2.0179 - val_accuracy: 0.2677 - val_loss: 3.5004
Epoch 4/10
205/205 ━━━━━━━━━━━━━━━━━━━━ 122s 597ms/step - accuracy: 0.8236 - loss: 0.6923 - val_accuracy: 0.2848 - val_loss: 4.6935
Epoch 5/10
205/205 ━━━━━━━━━━━━━━━━━━━━ 122s 593ms/step - accuracy: 0.9519 - loss: 0.1865 - val_accuracy: 0.2579 - val_loss: 5.0643
Epoch 6/10
205/205 ━━━━━━━━━━━━━━━━━━━━ 126s 617ms/step - accuracy: 0.9899 - loss: 0.0536 - val_accuracy: 0.2775 - val_loss: 5.5538
Epoch 7/10
205/205 ━━━━━━━━━━━━━━━━━━━━ 135s 661ms/step - accuracy: 0.9942 - loss: 0.0274 - val_accuracy: 0.2738 - val_loss: 5.4949
Epoch 8/10
205/205 ━━━━━━━━━━━━━━━━━━━━ 127s 620ms/step - accuracy: 0.9979 -

In [ ]:
# Salvando o modelo treinado no novo formato
model.save('modelo_flores.keras')
print("Modelo salvo com sucesso!")

import os
if os.path.exists('modelo_flores.keras'):
    print(f"Arquivo do modelo encontrado em: {os.path.abspath('modelo_flores.keras')}")
else:
    print("Erro: Arquivo do modelo não foi criado")

Modelo salvo com sucesso!
Arquivo do modelo encontrado em: d:\project_hub\Flower Recognization\notebooks\modelo_flores.keras
